# Cryptocurrency Time Series Analysis Notebook

## Objectives
- Load and preprocess cryptocurrency data
- Perform comprehensive time series analysis
- Visualize key trends and patterns
- Conduct statistical tests and decomposition

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical and time series analysis
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# Set plotting style
plt.style.use('seaborn')
sns.set_palette('deep')

## Data Loading and Preparation

In [ ]:
# Load processed cryptocurrency data
def load_crypto_data(crypto_name='bitcoin'):
    """
    Load processed cryptocurrency data
    """
    try:
        data = pd.read_csv(f'data/processed/features/{crypto_name}_engineered_features.csv', parse_dates=['timestamp'])
        data.set_index('timestamp', inplace=True)
        return data
    except FileNotFoundError:
        print(f"No data found for {crypto_name}. Ensure data preprocessing is complete.")
        return None

# Select cryptocurrency
crypto_name = 'bitcoin'
crypto_data = load_crypto_data(crypto_name)

# Display basic information
print(crypto_data.info())
print("\nFirst few rows:")
print(crypto_data.head())

## Time Series Stationarity Test

Augmented Dickey-Fuller test to check for stationarity

In [ ]:
def stationarity_test(timeseries):
    """
    Perform Augmented Dickey-Fuller test for stationarity
    """
    print('Results of Augmented Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

# Stationarity test on price
stationarity_test(crypto_data['price'])

## Autocorrelation and Partial Autocorrelation Analysis

In [ ]:
# Create ACF and PACF plots
plt.figure(figsize=(16,8))

# ACF Plot
plt.subplot(121)
plot_acf(crypto_data['price'].dropna(), lags=50, ax=plt.gca())
plt.title(f'{crypto_name.capitalize()} Price Autocorrelation')

# PACF Plot
plt.subplot(122)
plot_pacf(crypto_data['price'].dropna(), lags=50, ax=plt.gca())
plt.title(f'{crypto_name.capitalize()} Price Partial Autocorrelation')

plt.tight_layout()
plt.savefig(f'data/processed/analysis/{crypto_name}_autocorrelation.png')
plt.show()

## Seasonal Decomposition

In [ ]:
# Perform seasonal decomposition
decomposition = seasonal_decompose(crypto_data['price'], period=30)

# Plot decomposition
plt.figure(figsize=(16,12))

plt.subplot(411)
plt.title(f'{crypto_name.capitalize()} Time Series Decomposition')
plt.plot(decomposition.observed)
plt.ylabel('Observed')

plt.subplot(412)
plt.plot(decomposition.trend)
plt.ylabel('Trend')

plt.subplot(413)
plt.plot(decomposition.seasonal)
plt.ylabel('Seasonal')

plt.subplot(414)
plt.plot(decomposition.resid)
plt.ylabel('Residual')

plt.tight_layout()
plt.savefig(f'data/processed/analysis/{crypto_name}_decomposition.png')
plt.show()

## Rolling Window Analysis

In [ ]:
# Calculate rolling statistics
rolling_mean = crypto_data['price'].rolling(window=30).mean()
rolling_std = crypto_data['price'].rolling(window=30).std()

# Visualize rolling statistics
plt.figure(figsize=(16,6))
plt.plot(crypto_data['price'], label='Original Price')
plt.plot(rolling_mean, label='30-day Rolling Mean')
plt.fill_between(crypto_data.index, 
                 rolling_mean - 2*rolling_std, 
                 rolling_mean + 2*rolling_std, 
                 color='lightgray', alpha=0.3)
plt.title(f'{crypto_name.capitalize()} Price with Rolling Mean and Standard Deviation')
plt.legend()
plt.savefig(f'data/processed/analysis/{crypto_name}_rolling_stats.png')
plt.show()

## Price Volatility Analysis

In [ ]:
# Calculate daily returns
crypto_data['daily_return'] = crypto_data['price'].pct_change()

# Volatility visualization
plt.figure(figsize=(16,6))

plt.subplot(121)
crypto_data['daily_return'].hist(bins=50)
plt.title('Distribution of Daily Returns')
plt.xlabel('Daily Return')
plt.ylabel('Frequency')

plt.subplot(122)
sns.boxplot(crypto_data['daily_return'])
plt.title('Daily Returns Boxplot')
plt.xlabel('Daily Return')

plt.tight_layout()
plt.savefig(f'data/processed/analysis/{crypto_name}_volatility.png')
plt.show()

# Print volatility statistics
print("Volatility Statistics:")
print(crypto_data['daily_return'].describe())

## Technical Indicator Summary

In [ ]:
# Technical indicators summary
technical_columns = [
    'MA_7', 'MA_30', 'EMA_7', 'EMA_30', 
    'RSI_14', 'BB_Middle', 'BB_Upper', 'BB_Lower', 
    'MACD', 'Signal_Line'
]

# Summary statistics of technical indicators
technical_summary = crypto_data[technical_columns].describe()
print("Technical Indicators Summary:")
print(technical_summary)

# Save summary to CSV
technical_summary.to_csv(f'data/processed/analysis/{crypto_name}_technical_summary.csv')

## Key Insights and Conclusions

In [ ]:
# Generate comprehensive report
def generate_time_series_report():
    report = f"""
    Cryptocurrency Time Series Analysis Report for {crypto_name.capitalize()}
    ============================================================

    1. Stationarity Analysis
    - ADF Test Statistic: {stationarity_test(crypto_data['price'])}
    - Interpretation: [Analyze p-value and test statistic]

    2. Volatility Characteristics
    - Mean Daily Return: {crypto_data['daily_return'].mean():.4f}
    - Standard Deviation of Returns: {crypto_data['daily_return'].std():.4f}
    - Maximum Daily Return: {crypto_data['daily_return'].max():.4f}
    - Minimum Daily Return: {crypto_data['daily_return'].min():.4f}

    3. Technical Indicator Insights
    - 7-day Moving Average: {crypto_data['MA_7'].mean():.2f}
    - 30-day Moving Average: {crypto_data['MA_30'].mean():.2f}
    - RSI 14-day: {crypto_data['RSI_14'].mean():.2f}

    4. Trend and Seasonality
    - [Insights from seasonal decomposition]

    Recommendations:
    1. [Data-driven recommendation]
    2. [Risk management suggestion]
    3. [Further analysis recommendation]
    """
    
    with open(f'data/processed/analysis/{crypto_name}_analysis_report.txt', 'w') as f:
        f.write(report)
    
    print(report)

# Generate report
generate_time_series_report()